In [41]:
from scipy import optimize
from scipy.optimize import NonlinearConstraint
import numpy as np
from warnings import filterwarnings

filterwarnings("ignore")

constraint = NonlinearConstraint(lambda x: np.sum(np.abs(x)), 0, 1)


def sharpe_ratio(weights, signal_dist=[], kalshi_dist=[]):
    exp, exp_sq = 0, 0
    for i in range(len(weights)):
        v = 0
        for j in range(len(weights)):
            y, n = kalshi_dist[j], 1 - kalshi_dist[j]
            if j == i:
                if weights[j] > 0:
                    v += weights[j] * (1 - y)
                else:
                    v += abs(weights[j]) * -n
            else:
                if weights[j] > 0:
                    v += weights[j] * -y
                else:
                    v += abs(weights[j]) * (1 - n)
        exp += signal_dist[i] * v
        exp_sq += signal_dist[i] * v * v

    var = exp_sq - exp * exp
    return -(exp / np.sqrt(var))


signal_dist = [0.1, 0.3, 0.3, 0.2, 0.1]
kalshi_dist = [0.3, 0.4, 0.1, 0.1, 0.1]

x0 = np.zeros(len(signal_dist))
bounds = [(-1, 1) for _ in range(len(signal_dist))]
result = optimize.differential_evolution(
    sharpe_ratio,
    bounds,
    args=(signal_dist, kalshi_dist),
    constraints=[constraint],
    popsize=100,
    maxiter=1000,
)
print(np.sum(np.abs(result.x)))
print(list([round(float(x), 3) for x in result.x]))
print(-sharpe_ratio(result.x, signal_dist, kalshi_dist))

0.5000069514303054
[-0.299, -0.076, 0.058, 0.035, -0.032]
0.7852812659593166
